# Importación de datos

In [3]:
import os
import cv2
import numpy as np
import pathlib
import matplotlib.pyplot as plt

data_dir = pathlib.Path('assets')

# Lista para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Clases disponibles
clases = ['class_A', 'class_B', 'class_C',]

# Recorrer las carpetas de las clases
for plate in os.listdir(data_dir):
    for clase in clases:
        for file in os.listdir(data_dir / plate / 'resized' / clase):
            # Leer la imagen
            imagen = cv2.imread(str(data_dir / plate / 'resized' / clase / file), cv2.IMREAD_COLOR)

            # Agregar la imagen a la lista
            imagenes.append(imagen)
            # Agregar la etiqueta a la lista en el formato 0 o 1
            etiquetas.append(0 if clase == 'class_B' else 1)

# Convertir las listas de imágenes y etiquetas a arrays numpy
imagenes = np.array(imagenes)

etiquetas = np.array(etiquetas)

print(imagenes.shape)
print(etiquetas.shape)


(73300, 128, 128, 3)
(73300,)


In [ ]:
# # Submuestreo de la clase mayoritaria
# 
# # Contar el número de muestras de cada clase
# clase_0 = np.sum(etiquetas == 0)
# clase_1 = np.sum(etiquetas == 1)
# 
# # Determinar la clase mayoritaria y la clase minoritaria
# clase_mayoritaria = 0 if clase_0 > clase_1 else 1
# clase_minoritaria = 1 if clase_mayoritaria == 0 else 0
# 
# # Calcular la diferencia entre el número de muestras de la clase mayoritaria y la clase minoritaria
# diferencia = clase_0 - clase_1 if clase_0 > clase_1 else clase_1 - clase_0
# 
# # Encontrar los índices de las muestras de la clase mayoritaria
# indices = np.where(etiquetas == clase_mayoritaria)[0]
# 
# # Seleccionar aleatoriamente las muestras de la clase mayoritaria a eliminar
# indices = np.random.choice(indices, diferencia, replace=False)
# 
# # Eliminar las muestras de la clase mayoritaria
# imagenes = np.delete(imagenes, indices, axis=0)
# etiquetas = np.delete(etiquetas, indices, axis=0)
# 
# # Contar el número de muestras de cada clase después del submuestreo
# clase_0 = np.sum(etiquetas == 0)
# clase_1 = np.sum(etiquetas == 1)
# 
# print(clase_0, clase_1)
# 
# # etiquetas = tf.keras.utils.to_categorical(etiquetas, num_classes=2)
# 
# print(imagenes.shape)
# print(etiquetas.shape)
# 

In [4]:
# Oversampling de la clase minoritaria

# Contar el número de muestras de cada clase
clase_0 = np.sum(etiquetas == 0)
clase_1 = np.sum(etiquetas == 1)

# Determinar la clase mayoritaria y la clase minoritaria
clase_mayoritaria = 0 if clase_0 > clase_1 else 1
clase_minoritaria = 1 if clase_mayoritaria == 0 else 0

# Calcular la diferencia entre el número de muestras de la clase mayoritaria y la clase minoritaria
diferencia = clase_0 - clase_1 if clase_0 > clase_1 else clase_1 - clase_0

# Encontrar los índices de las muestras de la clase minoritaria
indices = np.where(etiquetas == clase_minoritaria)[0]

# Seleccionar aleatoriamente las muestras de la clase minoritaria a duplicar
indices = np.random.choice(indices, diferencia, replace=True)

# Duplicar las muestras de la clase minoritaria
imagenes = np.concatenate((imagenes, imagenes[indices]), axis=0)
etiquetas = np.concatenate((etiquetas, etiquetas[indices]), axis=0)

# Contar el número de muestras de cada clase después del sobremuestreo
clase_0 = np.sum(etiquetas == 0)
clase_1 = np.sum(etiquetas == 1)

print(clase_0, clase_1)

# Guardar las imágenes y etiquetas en un directorio
data_clase_A = 'data/clase_A'
data_clase_B = 'data/clase_B'

if not os.path.exists(data_clase_A):
    os.makedirs(data_clase_A)

if not os.path.exists(data_clase_B):
    os.makedirs(data_clase_B)

for i, (imagen, etiqueta) in enumerate(zip(imagenes, etiquetas)):
    clase = 'clase_A' if etiqueta == 1 else 'clase_B'
    cv2.imwrite(f'{data_clase_A if etiqueta == 1 else data_clase_B}/{clase}_{i}.jpg', imagen)


65889 65889


# Cargar ResNet50 de base

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

imagen_alto = 128
imagen_ancho = 128
canales_de_color = 3
num_clases = 2  # Número de clases

data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(imagen_alto, imagen_ancho, canales_de_color))
base_model.trainable = False

inputs = layers.Input(shape=(imagen_alto, imagen_ancho, canales_de_color))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(num_clases, activation='softmax')(x)  # Cambiado a softmax para clasificación múltiple

model = models.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,687,938 (97.99 MB)

 Trainable params: 2,100,226 (8.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

# Importar datos desde disco

In [2]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Parámetros
data_dir = 'data'
batch_size = 32
img_height = 128
img_width = 128

# Generador de entrenamiento
train_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # 80% entrenamiento, 20% validación
    subset="training",      # Usar el conjunto de entrenamiento
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
).map(lambda x, y: (x / 255.0, y))  # Normalizar las imágenes

# Generador de validación
val_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # 20% validación
    subset="validation",    # Usar el conjunto de validación
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
).map(lambda x, y: (x / 255.0, y))  # Normalizar las imágenes


Found 131778 files belonging to 2 classes.
Using 105423 files for training.
Found 131778 files belonging to 2 classes.
Using 26355 files for validation.
Epoch 1/10
3295/3295 ━━━━━━━━━━━━━━━━━━━━ 3019s 914ms/step - accuracy: 0.6574 - loss: 0.6240 - val_accuracy: 0.6886 - val_loss: 0.5938
Epoch 2/10
3295/3295 ━━━━━━━━━━━━━━━━━━━━ 2866s 870ms/step - accuracy: 0.6804 - loss: 0.6043 - val_accuracy: 0.6907 - val_loss: 0.5909
Epoch 3/10
3295/3295 ━━━━━━━━━━━━━━━━━━━━ 2856s 867ms/step - accuracy: 0.6839 - loss: 0.6010 - val_accuracy: 0.6967 - val_loss: 0.5808
Epoch 4/10
3295/3295 ━━━━━━━━━━━━━━━━━━━━ 2826s 858ms/step - accuracy: 0.6855 - loss: 0.5971 - val_accuracy: 0.6990 - val_loss: 0.5788
Epoch 5/10
3295/3295 ━━━━━━━━━━━━━━━━━━━━ 2994s 909ms/step - accuracy: 0.6886 - loss: 0.5929 - val_accuracy: 0.6972 - val_loss: 0.5786
Epoch 6/10
3295/3295 ━━━━━━━━━━━━━━━━━━━━ 3029s 919ms/step - accuracy: 0.6896 - loss: 0.5917 - val_accuracy: 0.7045 - val_loss: 0.5728
Epoch 7/10
3295/3295 ━━━━━━━━━━━━━━━━

# Entrenar el modelo

In [ ]:
# Entrenar el modelo con el generador de entrenamiento y validar en cada época
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset  # Usar validación para medir el desempeño
)

# Guardado del entrenamiento

In [5]:
from datetime import datetime

now = datetime.now().strftime("%Y%m%d%H%M%S")

model.save_weights(f'pesos_modelo_cnn_{now}.weights.h5')
model.save(filepath=f'modelo_cnn_{now}.keras')

In [38]:
# Make a sound when done

import winsound
frequency = 880  # Set Frequency To 2500 Hertz

duration = 700  # Set Duration To 1000 ms == 1 second

winsound.Beep(frequency, duration)


In [ ]:
# Mostrar la matriz de confusión
from sklearn.metrics import confusion_matrix
import seaborn
import pandas


clases = ['class_A', 'class_B']
# Obtener las predicciones del modelo
predicciones = model.predict(
    val_dataset,
    verbose=1
)[:, 1]

# Convertir las predicciones a etiquetas
predicciones = np.where(predicciones > 0.5, 1, 0)

# Calcular la matriz de confusión
cm = confusion_matrix(val_dataset.labels, predicciones)

# Crear un DataFrame de pandas con la matriz de confusión
df = pandas.DataFrame(cm, index=clases, columns=clases)


# Mostrar la matriz de confusión
seaborn.heatmap(df, annot=True, cmap='Blues')
plt.xlabel('Clase predicha')
plt.ylabel('Clase real')
plt.title('Matriz de confusión')
plt.show()
